## Echem

In [ ]:
# -*- coding: utf-8 -*-

import sys
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import gridspec, ticker
from matplotlib.colors import LinearSegmentedColormap

In [ ]:
# Ensure custom module Path is set before import
sys.path.append(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure")
from colors import tol_cmap, tol_cset  # type: ignore

# 画图的初始设置
plt.style.use(r"D:\CHENG\OneDrive - UAB\ICMAB-Python\Figure\liuchzzyy.mplstyle")
# print(plt.style.available)  # noqa: ERA001

# xarray setting
xr.set_options(
    cmap_sequential="viridis",
    cmap_divergent="viridis",
    display_width=150,
)  # viridis, gray

# 颜色设定
colors = tol_cset("vibrant")
if colors is not None:
    colors = list(colors)
else:
    # Fallback colors in case tol_cset returns None
    colors = ["#0077BB", "#33BBEE", "#009988", "#EE7733", "#CC3311", "#EE3377", "#BBBBBB"]
if r"sunset" not in plt.colormaps():
    cmap = tol_cmap("sunset")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)
if r"rainbow_PuRd" not in plt.colormaps():
    cmap = tol_cmap("rainbow_PuRd")
    if isinstance(cmap, LinearSegmentedColormap):
        plt.colormaps.register(cmap)  # 备用 plasma

# 输出的文件夹
path_out = Path(r"C:\Users\chengliu\Desktop\Figure")

# Set math font
mpl.rcParams["mathtext.fontset"] = "custom"
mpl.rcParams["mathtext.rm"] = "Arial"
mpl.rcParams["mathtext.it"] = "Arial:italic"
mpl.rcParams["mathtext.bf"] = "Arial:bold"
mpl.rcParams["mathtext.sf"] = "Arial"
mpl.rcParams["mathtext.tt"] = "Arial"
mpl.rcParams["mathtext.cal"] = "Arial"
mpl.rcParams["mathtext.default"] = "regular"

### Echem, EMD, 30 - 8

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\SEM\ExSitu\EMD\Pristine\Results\30mAh-10mA-8cm2-0.4M MnAC2-HAC-NaAC-pH4.25\Echem\15-01-2024-6cm2").glob( # noqa: E501, RUF001
        r"**\*.txt"
    )
)
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
        pd.to_numeric, errors="coerce"
    )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]  # noqa: E501

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
labels = [[r"$\mathrm{EMD \ MnO_2}$", None, None, None], [None, None, None, None]]
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx].reset_index(drop=True)
        # 找到电压最小值的索引
        idx_max = temp["Capacity/mA.h"].idxmax()
        # 断开最小值前后的曲线
        ax.plot(
            # temp.loc[:idx_max, "Capacity/mA.h"] * 1000 / mass[i],  # noqa: ERA001
            temp.loc[:idx_max, "Capacity/mA.h"]/8,
            temp.loc[:idx_max, "Ewe/V"],
            ls="-",
            lw=1.0,
            c=colors[i],
            label=labels[i][j],
            zorder=0,
        )
        ax.plot(
            # temp.loc[idx_max+10:, "Capacity/mA.h"] * 1000 / mass[i],  # noqa: ERA001
            temp.loc[idx_max+10:, "Capacity/mA.h"]/8,
            temp.loc[idx_max+10:, "Ewe/V"],
            ls="-",
            lw=1.0,
            c=colors[i],
            label=None,
            zorder=0,
        )
# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \ cm^{-2}_{MnO_2})}$", fontsize=11)
ax.set_xlim(0, 4)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.8, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.4, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.0, 0.4), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.text(
    0.03,
    0.15,
    r"$\mathrm{Bulk \ Cell, Buffered, 1.25 \ mA \ cm^{-2}}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{0.4M \ Mn(CH_3COO)_2}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"Ecehm_αMnO2_BulkCell_1MZn_0.2MMn_1_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"Ecehm_αMnO2_BulkCell_1MZn_0.2MMn_1_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor('white')
plt.show()

### Echem, alphaMnO2 + Bulk Cell + pH buffer

In [ ]:
path_filelist = list(
    Path(r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\Echem\αMnO2\SpecialTest\pHBuffer\Bulk Cell").glob(
        r"**\*.txt"
    )
)  # noqa: E501, RUF001
path_filelist

In [ ]:
# 读取电化学数据
echem = []
for path_file in path_filelist:
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
                break  # 发现后立即退出循环，提高效率

    df = pd.read_csv(
        path_file, sep="\t", comment="#", skiprows=line_skip - 1, encoding="latin_1", index_col=None, decimal="."
    ).dropna(axis=1, how="all")  # noqa: E501
    # # 转换数据格式
    df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = df[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
        pd.to_numeric, errors="coerce"
    )  # noqa: E501
    df["time/s"] = df["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    echem.append(df)

# 选取每个数据的第一到第二圈
for i in range(len(echem)):
    echem[i] = echem[i][echem[i].iloc[:, 0].isin([0,1])]  # noqa: E501

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)
labels = [[r"0.4 mg", None, None, None], [r"1.36 mg", None, None, None]]
mass = [0.4, 1.36]
capacity1st = []
for i, data in enumerate(echem):
    for j, idx in enumerate(data.iloc[:, 0].unique()):
        temp = data[data.iloc[:, 0] == idx].reset_index(drop=True)
        # 找到电压最小值的索引
        idx_max = temp["Capacity/mA.h"].idxmax()
        if j==0:
            # 断开最小值前后的曲线
            ax.plot(
                temp.loc[:idx_max, "Capacity/mA.h"] * 1000 / mass[i]/800 * 100,  # noqa: ERA001
                temp.loc[:idx_max, "Ewe/V"],
                ls="-",
                lw=1.0,
                c=colors[i],
                label=labels[i][j],
                zorder=0,
            )
        # 断开最小值前后的曲线
        if j == 1:
            data_max = temp["Capacity/mA.h"].max()
            ax.plot(
                # temp.loc[:idx_max, "Capacity/mA.h"] * 1000 / mass[i],  # noqa: ERA001
                temp.loc[:idx_max, "Capacity/mA.h"]*100 / data_max,
                temp.loc[:idx_max, "Ewe/V"],
                ls="-",
                lw=1.0,
                c=colors[i],
                label=labels[i][j],
                zorder=0,
            )
            ax.plot(
                # temp.loc[idx_max+10:, "Capacity/mA.h"] * 1000 / mass[i],  # noqa: ERA001
                temp.loc[idx_max+10:, "Capacity/mA.h"]*100 / data_max,
                temp.loc[idx_max+10:, "Ewe/V"],
                ls="-",
                lw=1.0,
                c=colors[i],
                label=None,
                zorder=0,
            )
# 设置刻度线等格式
ax.set_xlabel(r"Full Charge Percentage (%)", fontsize=11)
ax.set_xlim(0, 105)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=20, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=10, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=11)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper left", bbox_to_anchor=(0.0, 0.4), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.text(
    0.03,
    0.13,
    r"$\mathrm{Bulk \ Cell, Buffered, C/5}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.03,
    0.07,
    r"$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"Ecehm_αMnO2_BulkCell_1MZn_0.2MMn_1_300.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"Ecehm_αMnO2_BulkCell_1MZn_0.2MMn_1_600.tif"),
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor('white')
plt.show()

### SEM, Cycled0.9V

In [ ]:
# 电化学上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\SEM\ExSitu\αMnO2\Charge\Electrode\Cycled0.9V\BulkCell\Results\Echem\1M ZnSO4 + 0.2M MnSO4\LC-pH Buffer-1M ZnSO4 + 0.2M MnSO4-HAC-NaAC-4.19-αMnO2-12cm-80992\A-A"  # noqa: E501, RUF001
)

# 读取电化学数据
with open(path_file.joinpath(r"ACETATE BUFFE-NH3 free CVC_02_GCPL_C02.txt"), "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break

data = pd.read_csv(
    path_file.joinpath(r"ACETATE BUFFE-NH3 free CVC_02_GCPL_C02.txt"),
    sep=r"\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    index_col=None,
    decimal=".",
    engine="python",
).dropna(axis=1, how="all")  # noqa: E501
data.columns = [r"cycle number", r"time/s", r"Ewe/V", r"Ece/V", r"<I>/mA", r"Capacity/mA.h"]
# 转换数据格式
data[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = data[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
    pd.to_numeric, errors="coerce"
)  # noqa: E501
data["time/s"] = data["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
data["cycle number"] = data["cycle number"].astype(float).astype(np.int16)

data = data[data.iloc[:, 0].isin([30])]  # 只保留一圈数据, 最后一圈

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

labels = [r"Data", None, None, None]

for i, idx in enumerate(data.iloc[:, 0].unique()):
    temp = data[data.iloc[:, 0] == idx]
    # 找到电压最小值的索引
    idx_min = temp["Ewe/V"].idxmax()
    # 断开最小值前后的曲线
    ax.plot(
        temp.loc[idx_min + 5 :, "Capacity/mA.h"],
        temp.loc[idx_min + 5 :, "Ewe/V"],
        ls="-",
        lw=1.0,
        c=colors[i],
        label=labels[i],
        zorder=0,
    )  # noqa: E501
    ax.plot(
        temp.loc[:idx_min, "Capacity/mA.h"],
        temp.loc[:idx_min, "Ewe/V"],
        ls="-",
        lw=1.0,
        c=colors[i],
        label=None,
        zorder=0,
    )  # noqa: E501

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mA\ h_{MnO_2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 1.5)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.3, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.15, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=9)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 0.6), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.text(
    0.02,
    0.07,
    r"$\mathrm{1M \ ZnSO_4 + 0.2M \ MnSO_4, 0.8C}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)
ax.text(
    0.02,
    0.14,
    r"$\mathrm{Bulk \ Cell}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"Ecehm_αMnO2_0.5MZn_0.2MMn_1_300.tif"),  # noqa: RUF001
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"Ecehm_αMnO2_0.5MZn_0.2MMn_1_600.tif"),  # noqa: RUF001
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

### SEM, Operando XAS, Cell3

In [ ]:
# 电化学上的时间
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\PaperDos\SEM\ExSitu\αMnO2\Charge\Electrode\Cycled1.80V\CoinCell-afterOperandoXAS\Results\Echem"  # noqa: E501, RUF001
)

# 读取电化学数据
with open(path_file.joinpath(r"cell3_03C.txt"), "r", encoding="latin_1") as file:
    for line in file:
        if line.startswith("Nb header lines"):
            line_skip = int(line.split(":")[1].strip())
            break

data = pd.read_csv(
    path_file.joinpath(r"cell3_03C.txt"),
    sep=r"\t",
    comment="#",
    skiprows=line_skip - 1,
    encoding="latin_1",
    index_col=None,
    decimal=".",
    engine="python",
).dropna(axis=1, how="all")  # noqa: E501
data.columns = [r"cycle number", r"time/s", r"Ewe/V", r"Ece/V", r"<I>/mA", r"Capacity/mA.h"]
# 转换数据格式
data[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]] = data[["Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]].apply(
    pd.to_numeric, errors="coerce"
)  # noqa: E501
data["time/s"] = data["time/s"].apply(pd.to_datetime, format="mixed", errors="coerce")
data["cycle number"] = data["cycle number"].astype(float).astype(np.int16)

data = data[data.iloc[:, 0].isin([0])]  # 只保留第一圈数据

In [ ]:
%matplotlib inline
plt.close("all")

# 画图
fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 1, width_ratios=None, height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0])
ax = subfig.add_axes((0, 0, 1, 1))
ax.set_box_aspect(0.8)

labels = [r"Data", None, None, None]

for i, idx in enumerate(data.iloc[:, 0].unique()):
    temp = data[data.iloc[:, 0] == idx]
    # 找到电压最小值的索引
    idx_min = temp["Ewe/V"].idxmin()
    # 断开最小值前后的曲线
    ax.plot(
        temp.loc[: idx_min - 1, "Capacity/mA.h"] * 1000 / 1.064,
        temp.loc[: idx_min - 1, "Ewe/V"],
        ls="-",
        lw=1.0,
        c=colors[i],
        label=labels[i],
        zorder=0,
    )  # noqa: E501
    ax.plot(
        temp.loc[idx_min + 5 :, "Capacity/mA.h"] * 1000 / 1.064,
        temp.loc[idx_min + 5 :, "Ewe/V"],
        ls="-",
        lw=1.0,
        c=colors[i],
        label=None,
        zorder=0,
    )  # noqa: E501

# 设置刻度线等格式
ax.set_xlabel(r"$\mathrm{Capacity \ (mAh \,g^{-1}_{MnO_2})}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0, 400)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=80, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=40, offset=0))

ax.set_ylabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}}\!)$", fontsize=9)
ax.set_ylim(0.85, 1.85)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0.05))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0.05))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(loc="upper right", bbox_to_anchor=(1.0, 0.6), ncols=1, frameon=False, labelcolor="linecolor", fontsize=9)
ax.text(
    0.02,
    0.07,
    r"$\mathrm{0.5M \ ZnSO_4 + 0.2M \ MnSO_4, C/10}$",
    ha="left",
    va="top",
    transform=ax.transAxes,
    fontsize=9,
    c="k",
)

# 保存图片
plt.savefig(
    Path.joinpath(path_out, r"Ecehm_αMnO2_0.5MZn_0.2MMn_1_300.tif"),  # noqa: RUF001
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"Ecehm_αMnO2_0.5MZn_0.2MMn_1_600.tif"),  # noqa: RUF001
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    transparent=False,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

### 读取并且合并不同扫速的 CV 数据

In [ ]:
# 读取数据
path_file_list = list(
    Path(
        r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\Echem\Bacterial Cellulose\αMnO2\50PVDF20SP40αMnO2\CV\1M ZnSO4 + 0.2M MnSO4\Results"  # noqa: E501, RUF001
    ).glob("*.txt")
)

echem = {}
for path_file in path_file_list:
    line_skip = scan_rate = None
    with open(path_file, "r", encoding="latin_1") as file:
        for line in file:
            if line.startswith("Nb header lines"):
                line_skip = int(line.split(":")[1].strip())
            elif line.startswith("dE/dt"):
                scan_rate = float(line.split()[1])
            if line_skip and scan_rate is not None:
                break
    df = pd.read_csv(
        path_file,
        sep=r"\t",
        comment="#",
        skiprows=line_skip if line_skip is not None else 0,
        engine="python",
        encoding="latin_1",
        decimal=".",
        header=None,
        index_col=None,
    ).dropna(axis=1, how="all")  # noqa: E501
    df.columns = ["cycle number", "time/s", "Ewe/V", "Ece/V", "<I>/mA", "Capacity/mA.h"]  # noqa: E501
    df.drop(columns=["time/s", "Ece/V", "Capacity/mA.h"], inplace=True, errors="ignore")  # 删除不需要的列
    df[["Ewe/V", "<I>/mA"]] = df[["Ewe/V", "<I>/mA"]].apply(pd.to_numeric, errors="coerce")
    df["cycle number"] = df["cycle number"].astype(float).astype(np.int16)
    df = df[df["cycle number"].isin([1, 2])]  # 保留第1+1圈
    idx_min = df[df["cycle number"] == 1]["Ewe/V"].idxmax()  # noqa: PLR2004
    idx_max = df[df["cycle number"] == 2]["Ewe/V"].idxmax()  # noqa: PLR2004
    df = df.loc[idx_min:idx_max].reset_index(drop=True)  # noqa: PLR2004
    echem[f"{scan_rate} mV/s"] = df

In [ ]:
lengths = {k: v.shape for k, v in echem.items()}
for k, v in lengths.items():
    print(f"{k}: {v}")

# 获取所有 df 的 index 交集
common_index = None
for df in echem.values():
    common_index = df.index if common_index is None else common_index.intersection(df.index)

for key in echem:  # noqa: PLC0206
    echem[key] = echem[key].reindex(common_index)

current = pd.DataFrame(np.array(list(echem.values()))[:, :, 2]).T  # units: mA
voltage = pd.DataFrame(np.array(list(echem.values()))[:, :, 1]).T  # units: V

current.columns = [f"mA{i + 1}" for i in range(current.shape[1])]
voltage.columns = list(echem.keys())

# 按 interleave 交错顺序重新排列
interleaved_columns = [item for pair in zip(voltage.columns.to_list(), current.columns.to_list()) for item in pair]

results = pd.concat([voltage, current], axis=1, ignore_index=False)
results = results[interleaved_columns]  # 按照交错顺序重新排列
results.to_csv(path_out.joinpath("CV_results.csv"), index=False, header=True)

In [ ]:
%matplotlib widget
from scipy.signal import find_peaks, savgol_filter

num_peaks = 3

# 2. 遍历每一条电流曲线
for index, col in enumerate(current.columns):
    i_data = current[col].values

    # 1. 数据预处理（去除 NaN 和异常值）
    i_data = pd.Series(i_data).interpolate().fillna(0).values
    # 3. 平滑处理（使用 Savitzky-Golay 滤波器）
    smoothed = savgol_filter(i_data, window_length=11, polyorder=3)

    # 4. 正峰检测（氧化峰）
    peaks_pos, _ = find_peaks(smoothed, prominence=0.001)
    peak_vals_pos = smoothed[peaks_pos]
    top_indices_pos = peaks_pos[np.argsort(peak_vals_pos)[-num_peaks:][::-1]]

    # 5. 负峰检测（还原峰）
    peaks_neg, _ = find_peaks(-smoothed, prominence=0.001)
    peak_vals_neg = smoothed[peaks_neg]
    top_indices_neg = peaks_neg[np.argsort(-peak_vals_neg)[-num_peaks:][::-1]]

    # 6. 绘图
    plt.figure(figsize=(10, 4))
    plt.plot(i_data, label="Raw data", alpha=0.4)
    plt.plot(smoothed, label="Smoothed", linewidth=2)

    # 正峰标注
    plt.plot(top_indices_pos, smoothed[top_indices_pos], "ro", label="Oxidation Peaks")
    for i in top_indices_pos:
        plt.annotate(
            f"{smoothed[i]:.2f}",
            xy=(i, smoothed[i]),
            xytext=(i, smoothed[i] + 0.02),
            arrowprops={"arrowstyle": "->", "color": "red"},
            fontsize=8,
        )

    # 负峰标注
    plt.plot(top_indices_neg, smoothed[top_indices_neg], "bo", label="Reduction Peaks")
    for i in top_indices_neg:
        plt.annotate(
            f"{smoothed[i]:.2f}",
            xy=(i, smoothed[i]),
            xytext=(i, smoothed[i] - 0.02),
            arrowprops={"arrowstyle": "->", "color": "blue"},
            fontsize=8,
        )

    # 图形设置
    plt.title(f"Peak Detection for {col}")
    plt.xlabel("Index (a.u.)")
    plt.ylabel("Current (mA)")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

### CV 画图

In [ ]:
path_file = Path(
    r"D:\CHENG\OneDrive - UAB\ICMAB-Data\Zn-Mn\Results\Echem\Bacterial Cellulose\αMnO2\50PVDF20SP40αMnO2\CV\1M ZnSO4 + 0.2M MnSO4\Results"  # noqa: E501, RUF001
)  # noqa: E501, RUF001
data = pd.read_excel(
    path_file.joinpath("CV_50PVDF20SP40αMnO2.xlsx"),
    engine="openpyxl",
    sheet_name=0,
).dropna(axis=1, how="all")  # noqa: E501, RUF001
data = data.iloc[:, 26:]

In [ ]:
%matplotlib inline

plt.close("all")
scan_rate = [r"0.1 mV/s", r"0.3 mV/s", r"0.5 mV/s", r"0.7 mV/s", r"1.0 mV/s"]

fig = plt.figure(figsize=(3.3, 2.5))
gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

# 图
subfig = fig.add_subfigure(gs[0, 0], zorder=0)
ax = subfig.add_subplot()
ax.set_position((0, 0, 1.0, 1.0))
ax.set_box_aspect(0.8)

for i in range(data.shape[1] // 4):
    ax.plot(
        data.iloc[:, i * 4],
        data.iloc[:, i * 4 + 1],
        color=colors[i],
        label=scan_rate[i],
        linewidth=1.0,
    )

# 设置刻度线等格式
ax.set_ylabel(r"$\mathrm{Current \ (mA)}$", fontsize=11, labelpad=1.0)
ax.set_ylim(-0.8, 1.6)
ax.yaxis.set_major_locator(ticker.MultipleLocator(base=0.6, offset=-0.2))
ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=0.4, offset=-0.2))

ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
ax.set_xlim(0.8, 2.0)
ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0))

ax.tick_params(axis="both", direction="out", labelsize=9)
ax.legend(
    loc="upper left",
    bbox_to_anchor=(-0.08, 1.0),
    ncols=2,
    handlelength=0.0,
    frameon=False,
    labelcolor="linecolor",
    fontsize=8,
    columnspacing=0.3,
)

plt.savefig(
    Path.joinpath(path_out, r"MnO2_CV_all_300.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=300,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.savefig(
    Path.joinpath(path_out, r"MnO2_CV_all_600.tif"),
    transparent=False,
    pad_inches=0.05,
    bbox_inches="tight",
    dpi=600,
    pil_kwargs={"compression": "tiff_lzw"},
)
plt.gcf().set_facecolor("white")
plt.show()

In [ ]:
%matplotlib inline
ylims_params = [
    (-0.2, 0.3, 0.1, 0.05, 0, 0),
    (-0.3, 0.7, 0.2, 0.1, 0.1, 0.1),
    (-0.5, 1.0, 0.3, 0.15, 0.1, 0.1),
    (-0.6, 1.2, 0.6, 0.3, 0, 0),
    (-0.8, 1.6, 0.6, 0.3, -0.2, -0.2),
]

for i in range(data.shape[1] // 4):
    # 计算贡献
    area_all = np.trapz(data.iloc[:, i * 4 + 1], data.iloc[:, i * 4])
    area_capacitance = np.trapz(data.iloc[:, i * 4 + 2], data.iloc[:, i * 4])
    ratio = 100 * area_capacitance / area_all if area_all != 0 else 0

    fig = plt.figure(figsize=(3.3, 2.5))
    gs = gridspec.GridSpec(1, 2, width_ratios=[1, 1], height_ratios=None, wspace=0, hspace=0, figure=fig)

    # 图
    subfig = fig.add_subfigure(gs[0, 0], zorder=0)
    ax = subfig.add_subplot()
    ax.set_position((0, 0, 1.0, 1.0))
    ax.set_box_aspect(0.8)

    ax.plot(
        data.iloc[:, i * 4],
        data.iloc[:, i * 4 + 1],
        color=colors[0],
        label="Data",
        linewidth=1.0,
    )
    ax.plot(
        data.iloc[:, i * 4],
        data.iloc[:, i * 4 + 2],
        color=colors[1],
        label="Capacitance",
        linewidth=1.0,
    )

    # 设置刻度线等格式
    ax.set_ylabel(r"$\mathrm{Current \ (mA)}$", fontsize=11, labelpad=1.0)
    ax.set_ylim(ylims_params[i][0], ylims_params[i][1])
    ax.yaxis.set_major_locator(ticker.MultipleLocator(base=ylims_params[i][2], offset=ylims_params[i][4]))
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(base=ylims_params[i][3], offset=ylims_params[i][5]))

    ax.set_xlabel(r"Voltage (V vs. Zn/Zn$\mathrm{^{2\!+}\!)}$", fontsize=11, labelpad=1.0)
    ax.set_xlim(0.8, 2.0)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(base=0.2, offset=0))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(base=0.1, offset=0))

    ax.tick_params(axis="both", direction="out", labelsize=9)
    ax.legend(
        loc="upper left",
        bbox_to_anchor=(-0.01, 1.0),
        ncols=1,
        handlelength=0.8,
        frameon=False,
        labelcolor="linecolor",
        fontsize=9,
    )
    ax.text(
        0.05, 0.55, f"Ratio {ratio:.1f} %", transform=ax.transAxes, fontsize=9, color=colors[1], ha="left", va="top"
    )

    plt.savefig(
        Path.joinpath(path_out, f"MnO2_CV_{i}_300.tif"),
        transparent=False,
        pad_inches=0.05,
        bbox_inches="tight",
        dpi=300,
        pil_kwargs={"compression": "tiff_lzw"},
    )
    plt.savefig(
        Path.joinpath(path_out, f"MnO2_CV_{i}_600.tif"),
        transparent=False,
        pad_inches=0.05,
        bbox_inches="tight",
        dpi=600,
        pil_kwargs={"compression": "tiff_lzw"},
    )
plt.gcf().set_facecolor("white")
plt.show()